# Импорт библиотек

In [37]:
import mlflow
from mlflow.tracking import MlflowClient

import os

import pandas as pd
import numpy as np

from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder

from transformers import pipeline
from transformers import Trainer, TrainingArguments, EarlyStoppingCallback, AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding

from datasets import Dataset

import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.metrics import *

import joblib
import cloudpickle

import nltk
from nltk.tokenize import word_tokenize

nltk.download('stopwords')

from nltk.corpus import stopwords
from functools import lru_cache

from pymorphy3 import MorphAnalyzer
import re
import emoji
import torch

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Smart\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Настройка конфигураций

In [38]:
from omegaconf import OmegaConf
import os

def load_config(config_name):
    """Загрузка конфигурации для конкретного эксперимента"""
    config_path = f"configs/{config_name}.yml"
    
    if not os.path.exists(config_path):
        raise FileNotFoundError(f"Конфигурационный файл {config_path} не найден")
    
    # Загружаем основной конфиг
    cfg = OmegaConf.load(config_path)
    
    # Переопределяем переменными окружения
    if 'MLFLOW_TRACKING_URI' in os.environ:
        cfg.mlflow.tracking_uri = os.environ['MLFLOW_TRACKING_URI']
    
    return cfg

# Эксперименты с датасетами

## Первый эксперимент (Лемматизация, удаление знаков пунктуации)

In [39]:
#!   mlflow server --host 127.0.0.1 --port 8080

In [40]:
cfg = load_config("base")  # или другое имя вашего конфига

# Теперь можно использовать
mlflow.set_tracking_uri(cfg.mlflow.tracking_uri)

print(f"Tracking URI: {cfg.mlflow.tracking_uri}")

Tracking URI: http://127.0.0.1:8080


In [41]:
# Загружаем конфигурацию
cfg = load_config("preprocess_first")

# Инициализация компонентов
analyzer = MorphAnalyzer(lang='ru')

# Получаем стоп-слова из конфигурации
stop_words = nltk.corpus.stopwords.words('russian')
stop_words_cleaned = [
    w for w in stop_words
    if w not in cfg.preprocess.keep_words  # Теперь это должно работать
]

@lru_cache(maxsize=cfg.preprocess.lru_cache_size)
def lemmatization(text):
    return analyzer.parse(text)[0].normal_form

def preprocess_text(text):
    text = re.sub(cfg.preprocess.regex.remove_newlines, " ", text)

    # 2. Фиксируем кривые слова nчипсы → чипсы
    text = re.sub(cfg.preprocess.regex.fix_mistyped_n, r"\1", text)

    # 3. Убираем символы
    text = re.sub(cfg.preprocess.regex.remove_symbols, "", text)

    # 4. Чистим двойные пробелы
    text = re.sub(cfg.preprocess.regex.collapse_spaces, " ", text).strip()

    # 5. Токенизация
    tokens = nltk.word_tokenize(text)
    result = []

    for token in tokens:
        if token not in stop_words_cleaned:
            result.append(lemmatization(token))

    return " ".join(result)

# Проверка работы
test_text = "Всем привет! Какое же неприятное место, нет?\n"
processed = preprocess_text(test_text)
print(f"Исходный: {test_text}")
print(f"Обработанный: {processed}")

Исходный: Всем привет! Какое же неприятное место, нет?

Обработанный: весь привет какой неприятный место нет


In [42]:
with mlflow.start_run(run_name="First dataset"):

    mlflow.set_tag("Dataset_version", cfg.mlflow.dataset_version)

    # Загружаем все файлы из конфига
    annotation_dfs = [
        pd.read_json(path) for path in cfg.preprocess.input_files
    ]

    df_annotations = pd.concat(annotation_dfs)
    
    df = pd.DataFrame(columns=["span", "label"])

    for mark in df_annotations['aspect_sentiment']:
        for entry in mark:
            span = entry['text']
            label = entry['labels'][0]
            df.loc[len(df)] = [span, label]

    # Применяем препроцессинг
    df['span'] = df['span'].apply(preprocess_text)

    display(df)

    # Сохраняем датасет
    df.to_csv(cfg.preprocess.output.dataset_csv, index=False)

    mlflow.log_artifact(cfg.preprocess.output.dataset_csv, "datasets")

    # Сохраняем препроцесс-функцию
    with open(cfg.preprocess.output.preprocess_pickle, "wb") as f:
        cloudpickle.dump(preprocess_text, f)

    mlflow.log_artifact(cfg.preprocess.output.preprocess_pickle, "functions")
    

,span,label
0,вкус реально отличный,ВКУС_POSITIVE
1,я есть чипсы очень долго год 2 назад советоват...,O
2,хотеться купить очень последний время стать оч...,O
3,вкус иметь каждый 2 пачка,O
4,вкус который стать намного острый обычный,ВКУС_NEGATIVE
...,...,...
2503,ярко красиво фиолетовый цвет любить новогодний...,ПАЧКА_POSITIVE
2504,чипсы довольно жирный не критично заметно,ТЕКСТУРА_NEUTRAL
2505,сам чипсы не идеально круглый,ТЕКСТУРА_NEUTRAL
2506,красивый золотистый нет пригорелый поломать ми...,ТЕКСТУРА_POSITIVE


🏃 View run First dataset at: http://127.0.0.1:8080/#/experiments/0/runs/d63ca4ae2b054f49b3ce30a18216827a
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/0


## Второй эксперимент (Простая очистка от знаков препинания и изменение эмодзи)

In [43]:
from functools import partial

cfg = load_config('preprocess_second')

def clean_text_only(text, cfg=None):
    """
    Очистка текста с поддержкой конфигурации
    
    Args:
        text: Входной текст
        cfg: Конфигурация (опционально)
    """
    if not isinstance(text, str):
        return ""
    
    # Если конфигурация не передана, используем значения по умолчанию
    lowercase = getattr(cfg, 'clean_only', {}).get('lowercase', True) if cfg else True
    replace_emoji = getattr(cfg, 'clean_only', {}).get('replace_emoji', True) if cfg else True
    remove_punctuation = getattr(cfg, 'clean_only', {}).get('remove_punctuation', True) if cfg else True
    remove_special_chars = getattr(cfg, 'clean_only', {}).get('remove_special_chars', True) if cfg else True
    collapse_spaces = getattr(cfg, 'clean_only', {}).get('collapse_spaces', True) if cfg else True
    
    if lowercase:
        text = text.lower()
    
    # Заменяем эмодзи
    if replace_emoji:
        text = emoji.demojize(text, delimiters=(" ", " "))
    
    # Удаляем переносы и спецсимволы
    if remove_special_chars:
        text = re.sub(r'[\n\r\t]', ' ', text)
    
    # Удаляем пунктуацию
    if remove_punctuation:
        text = re.sub(r'[^\w\s]', ' ', text)
    
    # Убираем лишние пробелы
    if collapse_spaces:
        text = re.sub(r'\s+', ' ', text).strip()
    
    return text

# Версия для обратной совместимости (без конфигурации)
def clean_text_only_legacy(text):
    """Легаси версия для обратной совместимости"""
    return clean_text_only(text)


with mlflow.start_run(run_name='Second dataset'):
    
    mlflow.set_tag("Dataset_version", cfg.mlflow.dataset_version)

    # Загружаем все файлы из конфига
    annotation_dfs = [
        pd.read_json(path) for path in cfg.preprocess.input_files
    ]

    df_annotations = pd.concat(annotation_dfs)
    
    df = pd.DataFrame(columns=["span", "label"])

    for mark in df_annotations['aspect_sentiment']:
        for entry in mark:
            span = entry['text']
            label = entry['labels'][0]
            df.loc[len(df)] = [span, label]

    # ИСПРАВЛЕНИЕ: передаем cfg в функцию препроцессинга
    # Вариант 1: Используем lambda
    df['span'] = df['span'].apply(lambda x: clean_text_only(x, cfg))
    
    # Или Вариант 2: Создаем частично примененную функцию
    # clean_text_with_config = partial(clean_text_only, cfg=cfg)
    # df['span'] = df['span'].apply(clean_text_with_config)

    display(df)

    # Сохраняем датасет
    df.to_csv(cfg.preprocess.output.dataset_csv, index=False)

    mlflow.log_artifact(cfg.preprocess.output.dataset_csv, "datasets")

    # ИСПРАВЛЕНИЕ: сохраняем функцию с привязанным конфигом
    # Создаем частично примененную функцию для сохранения
    clean_text_with_config = partial(clean_text_only, cfg=cfg)
    with open(cfg.preprocess.output.preprocess_pickle, "wb") as f:
        cloudpickle.dump(clean_text_with_config, f)

    mlflow.log_artifact(cfg.preprocess.output.preprocess_pickle, "functions")

,span,label
0,вкус был реально отличный,ВКУС_POSITIVE
1,я ел эти чипсы очень долго еще года 2 назад со...,O
2,хотелось купить очень много но в последнее вре...,O
3,сейчас такой вкус имеет каждая 2 пачка,O
4,вкус которых стал намного острее обычного,ВКУС_NEGATIVE
...,...,...
2503,ярко красиво фиолетовый цвет я люблю новогодне...,ПАЧКА_POSITIVE
2504,чипсы довольно жирные не критично но заметно,ТЕКСТУРА_NEUTRAL
2505,сами чипсы не идеально круглые,ТЕКСТУРА_NEUTRAL
2506,красивые золотистые нет пригорелых поломанных ...,ТЕКСТУРА_POSITIVE


🏃 View run Second dataset at: http://127.0.0.1:8080/#/experiments/0/runs/38adc29950984b408e94b3a3b5c0cb6c
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/0


## Третий эксперимент (Аугментация данных для минорных классов)

In [44]:
with mlflow.start_run(run_name="Third dataset"):

    cfg = load_config("preprocess_third")

    mlflow.set_tag("Dataset_version", cfg.mlflow.dataset_version)


    annotation_dfs = [pd.read_json(path) for path in cfg.preprocess.input_files]
    df_annotations = pd.concat(annotation_dfs, ignore_index=True)


    df = pd.DataFrame(columns=["span", "label"])

    for mark in df_annotations['aspect_sentiment']:
        for entry in mark:
            span = entry['text']
            label = entry['labels'][0]
            df.loc[len(df)] = [span, label]


    class_counts = df['label'].value_counts()
    minority_classes = class_counts[class_counts < class_counts.mean()].index.tolist()

    print(f"Minority classes: {minority_classes}")


    import random

    synonyms = cfg.preprocess.augmentation.synonyms
    n_variants = cfg.preprocess.augmentation.n_variants

    def simple_augmentation(text):
        augmented = []
        for _ in range(n_variants):
            words = text.split()
            new_words = []
            for w in words:
                wl = w.lower()
                if wl in synonyms and random.random() > 0.7:
                    new_words.append(random.choice(synonyms[wl]))
                else:
                    new_words.append(w)
            augmented.append(" ".join(new_words))
        return augmented


    augmented_data = []
    for label in minority_classes:
        samples = df[df['label'] == label]
        for _, row in samples.iterrows():
            for aug_text in simple_augmentation(row["span"]):
                augmented_data.append({"span": aug_text, "label": label})

    df_augmented = pd.DataFrame(augmented_data)


    df_extended = pd.concat([df, df_augmented], ignore_index=True)


    output_path = cfg.preprocess.output.dataset_csv
    df_extended.to_csv(output_path, index=False)

    mlflow.log_artifact(output_path, "datasets")

    print(f"Сохранен датасет с аугментацией: {output_path}")


Minority classes: ['ПАЧКА_POSITIVE', 'ТЕКСТУРА_NEUTRAL', 'ПАЧКА_NEUTRAL', 'ТЕКСТУРА_NEGATIVE', 'ПАЧКА_NEGATIVE']
Сохранен датасет с аугментацией: third_experiment_dataset.csv
🏃 View run Third dataset at: http://127.0.0.1:8080/#/experiments/0/runs/3f371b6b84564621b3f1f656172c4f72
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/0


# Эксперементирование с моделями

## Первый эксперимент (Простая модель)

Будем обучать простую модель: "Наивный Байесовский Классификатор" из `Sklearn`, будем проводить тест на 3 версиях датасета и поймем, какая модель лучше себя покажет при работе с тем или иным датасетом

### Первый датасет

In [ ]:
cfg = load_config("naive_bayes_first")

with mlflow.start_run(run_name='first_model_experiment'):

    mlflow.set_tag('NaiveBayes', cfg.model.version)

    # =====================================================================================================================================
    #                                         СЧИТЫВАНИЕ ДАТАСЕТА
    # =====================================================================================================================================

    client = MlflowClient()

    dataset_runs = client.search_runs(
        experiment_ids=[cfg.mlflow.experiment_id],
        filter_string=f"tags.mlflow.runName = '{cfg.data.source_run}'",
        order_by=['attributes.end_time desc']
    )

    first_dataset_latest_run = dataset_runs[0]
    first_dataset_latest_run_id = first_dataset_latest_run.info.run_id

    # Используем пути из конфига
    full_dataset_path = f"{cfg.data.dataset_path}/{cfg.data.dataset_file}"
    
    try:
        dataframe_path = client.download_artifacts(first_dataset_latest_run_id, full_dataset_path)
        df = pd.read_csv(dataframe_path)
        print(f"✅ Успешно загружен: {full_dataset_path}")
    except Exception as e:
        print(f"❌ Не удалось загрузить {full_dataset_path}: {e}")

        try:
            dataframe_path = client.download_artifacts(first_dataset_latest_run_id, cfg.data.dataset_file)
            df = pd.read_csv(dataframe_path)
            print(f"✅ Успешно загружен: {cfg.data.dataset_file}")
        except:
            raise ValueError(f"Не удалось загрузить датасет: {full_dataset_path}")
    
    display(df)

    # =====================================================================================================================================
    #                                              ВЕКТОРИЗАЦИЯ ТЕКСТА
    # =====================================================================================================================================

    vectorizer = TfidfVectorizer(
        lowercase=cfg.vectorizer.lowercase,
        analyzer=cfg.vectorizer.analyzer,
        max_features=cfg.vectorizer.max_features,
        ngram_range=tuple(cfg.vectorizer.ngram_range),
        min_df=cfg.vectorizer.min_df,
        max_df=cfg.vectorizer.max_df
    )
    
    encoder = LabelEncoder()

    # =====================================================================================================================================
    #                                         РАЗБИЕНИЕ НА TRAIN/TEST
    # =====================================================================================================================================

    df_train, df_test = train_test_split(
        df, 
        test_size=cfg.training.test_size, 
        random_state=cfg.training.random_state, 
        stratify=df['label']
    )

    X_train = df_train['span']
    X_train = vectorizer.fit_transform(X_train)

    y_train = df_train['label']
    y_train = encoder.fit_transform(y_train)

    X_test = df_test['span']
    X_test = vectorizer.transform(X_test)

    y_test = df_test['label']
    y_test = encoder.transform(y_test)

    # =====================================================================================================================================
    #                                         ОБУЧЕНИЕ И ЛОГИРОВАНИЕ МОДЕЛИ
    # =====================================================================================================================================


    mlflow.sklearn.autolog(disable=True)
    
    model = MultinomialNB()

    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    f1 = f1_score(y_test, y_pred, average='weighted')
    accuracy = accuracy_score(y_test, y_pred)

    print(f'f1-score равен {f1:.4f}')
    print(f'accuracy-score равен {accuracy:.4f}')

    # Сохраняем артефакты
    joblib.dump(model, cfg.model.artifacts.model)
    joblib.dump(vectorizer, cfg.model.artifacts.vectorizer)
    joblib.dump(encoder, cfg.model.artifacts.encoder)
    
    mlflow.log_artifact(cfg.model.artifacts.model, 'models')
    mlflow.log_artifact(cfg.model.artifacts.vectorizer, 'models')
    mlflow.log_artifact(cfg.model.artifacts.encoder, 'models')
    
    # Логируем кастомные метрики
    mlflow.log_metrics({
        'f1_score': f1,
        'accuracy': accuracy
    })

    # Логируем параметры из конфига
    mlflow.log_params({
        'vectorizer_max_features': cfg.vectorizer.max_features,
        'vectorizer_ngram_range': str(cfg.vectorizer.ngram_range),
        'test_size': cfg.training.test_size
    })

    print("Модель успешно обучена и залогирована!")

✅ Успешно загружен: datasets/first_experiment_dataset.csv


,span,label
0,вкус реально отличный,ВКУС_POSITIVE
1,я есть чипсы очень долго год 2 назад советоват...,O
2,хотеться купить очень последний время стать оч...,O
3,вкус иметь каждый 2 пачка,O
4,вкус который стать намного острый обычный,ВКУС_NEGATIVE
...,...,...
2503,ярко красиво фиолетовый цвет любить новогодний...,ПАЧКА_POSITIVE
2504,чипсы довольно жирный не критично заметно,ТЕКСТУРА_NEUTRAL
2505,сам чипсы не идеально круглый,ТЕКСТУРА_NEUTRAL
2506,красивый золотистый нет пригорелый поломать ми...,ТЕКСТУРА_POSITIVE


f1-score равен 0.3869
accuracy-score равен 0.4422
Модель успешно обучена и залогирована!
🏃 View run first_model_experiment at: http://127.0.0.1:8080/#/experiments/0/runs/6251fed9027b47df841d5093c07414b8
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/0


#### Проверка результатов

In [ ]:
# Загружаем конфиг для инференса
cfg_inference = load_config("inference_bayes_first")

client = MlflowClient()

# ===========================================================================================
# ПОЛУЧЕНИЕ МОДЕЛЕЙ
# ===========================================================================================


latest_run_model = client.search_runs(
    experiment_ids=[cfg_inference.mlflow.experiment_id],
    filter_string=f'tags.mlflow.runName = "{cfg_inference.model.run_name}"',
    order_by=['attributes.end_time desc']
)

latest_run_model_id = latest_run_model[0].info.run_id
print(f"Run ID модели: {latest_run_model_id}")


vectorizer_path = f"{cfg_inference.model.artifacts_path}/{cfg_inference.model.vectorizer}"
bayes_path = f"{cfg_inference.model.artifacts_path}/{cfg_inference.model.bayes}" 
encoder_path = f"{cfg_inference.model.artifacts_path}/{cfg_inference.model.encoder}"

try:
    vectorizer_file = client.download_artifacts(latest_run_model_id, vectorizer_path)
    bayes_file = client.download_artifacts(latest_run_model_id, bayes_path)
    encoder_file = client.download_artifacts(latest_run_model_id, encoder_path)
    print("✅ Модели успешно загружены")
except Exception as e:
    print(f"❌ Ошибка загрузки моделей: {e}")

    try:
        vectorizer_file = client.download_artifacts(latest_run_model_id, cfg_inference.model.vectorizer)
        bayes_file = client.download_artifacts(latest_run_model_id, cfg_inference.model.bayes)
        encoder_file = client.download_artifacts(latest_run_model_id, cfg_inference.model.encoder)
        print("✅ Модели загружены (fallback)")
    except:
        raise ValueError("Не удалось загрузить модели")

# ===========================================================================================
# ПОЛУЧЕНИЕ ФУНКЦИИ ПРЕПРОЦЕССИНГА
# ===========================================================================================


latest_run_dataset = client.search_runs(
    experiment_ids=[cfg_inference.mlflow.experiment_id],
    filter_string=f'tags.mlflow.runName = "{cfg_inference.preprocess.run_name}"',
    order_by=['attributes.end_time desc']
)

latest_run_dataset_id = latest_run_dataset[0].info.run_id

try:
    art_loc = client.download_artifacts(latest_run_dataset_id, cfg_inference.preprocess.artifact_path)
    print("✅ Функция препроцессинга загружена")
except Exception as e:
    print(f"❌ Ошибка загрузки функции: {e}")
    raise

# ===========================================================================================
# ЗАГРУЗКА И ИСПОЛЬЗОВАНИЕ МОДЕЛЕЙ
# ===========================================================================================


vectorizer = joblib.load(vectorizer_file)
bayes = joblib.load(bayes_file)
encoder = joblib.load(encoder_file)


with open(art_loc, 'rb') as f:
    preprocess_func = cloudpickle.load(f)

# ===========================================================================================
# ПРЕДСКАЗАНИЕ
# ===========================================================================================

# Используем текст из конфига
text = cfg_inference.test_text

preprocessed_text = preprocess_func(text)
done_text = vectorizer.transform([preprocessed_text])
label = bayes.predict(done_text)

print('=' * 100)
print(f'Исходный текст: "{text}"')
print(f'Предобработанный текст: "{preprocessed_text}"')
print(f'Предсказанная метка: {encoder.inverse_transform(label)[0]}')

Run ID модели: 6251fed9027b47df841d5093c07414b8


✅ Модели успешно загружены


✅ Функция препроцессинга загружена
Исходный текст: "Меня привлекло такое сочетание"
Предобработанный текст: "я привлечь такой сочетание"
Предсказанная метка: ВКУС_POSITIVE


### Второй датасет

In [ ]:

cfg = load_config("naive_bayes_second")

with mlflow.start_run(run_name='second_model_experiment'):

    mlflow.set_tag('NaiveBayes', cfg.model.version)

    # =====================================================================================================================================
    #                                         СЧИТЫВАНИЕ ДАТАСЕТА
    # =====================================================================================================================================

    client = MlflowClient()

    dataset_runs = client.search_runs(
        experiment_ids=[cfg.mlflow.experiment_id],
        filter_string=f"tags.mlflow.runName = '{cfg.data.source_run}'",
        order_by=['attributes.end_time desc']
    )

    first_dataset_latest_run = dataset_runs[0]
    first_dataset_latest_run_id = first_dataset_latest_run.info.run_id


    try:
        dataframe_path = client.download_artifacts(first_dataset_latest_run_id, "datasets/second_experiment_dataset.csv")
        df = pd.read_csv(dataframe_path)
        print("✅ Датасет загружен напрямую")
    except:
        try:
            files = client.list_artifacts(first_dataset_latest_run_id, cfg.data.dataset_path)
            dataframe = files[0].path
            dataframe_path = client.download_artifacts(first_dataset_latest_run_id, dataframe)
            df = pd.read_csv(dataframe_path)
            print("✅ Датасет загружен через list_artifacts")
        except:
            raise ValueError("Не удалось загрузить датасет")

    display(df)

    # =====================================================================================================================================
    #                                              ВЕКТОРИЗАЦИЯ ТЕКСТА
    # =====================================================================================================================================

    vectorizer = TfidfVectorizer(
        lowercase=cfg.vectorizer.lowercase,
        analyzer=cfg.vectorizer.analyzer,
        max_features=cfg.vectorizer.max_features,
        ngram_range=tuple(cfg.vectorizer.ngram_range),
        min_df=cfg.vectorizer.min_df,
        max_df=cfg.vectorizer.max_df
    )
    
    encoder = LabelEncoder()

    # =====================================================================================================================================
    #                                         РАЗБИЕНИЕ НА TRAIN/TEST
    # =====================================================================================================================================

    df_train, df_test = train_test_split(
        df, 
        test_size=cfg.training.test_size, 
        random_state=cfg.training.random_state, 
        stratify=df['label']
    )

    X_train = df_train['span']
    X_train = vectorizer.fit_transform(X_train)

    y_train = df_train['label']
    y_train = encoder.fit_transform(y_train)

    X_test = df_test['span']
    X_test = vectorizer.transform(X_test)

    y_test = df_test['label']
    y_test = encoder.transform(y_test)

    # =====================================================================================================================================
    #                                         ОБУЧЕНИЕ И ЛОГИРОВАНИЕ МОДЕЛИ
    # =====================================================================================================================================

    mlflow.sklearn.autolog(disable=True)

    model = MultinomialNB()

    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    f1 = f1_score(y_test, y_pred, average='weighted')
    accuracy = accuracy_score(y_test, y_pred)

    print(f'f1-score равен {f1:.4f}')
    print(f'accuracy-score равен {accuracy:.4f}')

    joblib.dump(model, cfg.model.artifacts.model)
    joblib.dump(vectorizer, cfg.model.artifacts.vectorizer)
    joblib.dump(encoder, cfg.model.artifacts.encoder)
    
    mlflow.log_artifact(cfg.model.artifacts.model, 'models')
    mlflow.log_artifact(cfg.model.artifacts.vectorizer, 'models')
    mlflow.log_artifact(cfg.model.artifacts.encoder, 'models')
    
    mlflow.log_metrics({
        'f1_score': f1,
        'accuracy': accuracy
    })

    mlflow.log_params({
        'vectorizer_max_features': cfg.vectorizer.max_features,
        'vectorizer_ngram_range': str(cfg.vectorizer.ngram_range),
        'test_size': cfg.training.test_size
    })

    print("Вторая модель успешно обучена и залогирована!")

✅ Датасет загружен напрямую


,span,label
0,вкус был реально отличный,ВКУС_POSITIVE
1,я ел эти чипсы очень долго еще года 2 назад со...,O
2,хотелось купить очень много но в последнее вре...,O
3,сейчас такой вкус имеет каждая 2 пачка,O
4,вкус которых стал намного острее обычного,ВКУС_NEGATIVE
...,...,...
2503,ярко красиво фиолетовый цвет я люблю новогодне...,ПАЧКА_POSITIVE
2504,чипсы довольно жирные не критично но заметно,ТЕКСТУРА_NEUTRAL
2505,сами чипсы не идеально круглые,ТЕКСТУРА_NEUTRAL
2506,красивые золотистые нет пригорелых поломанных ...,ТЕКСТУРА_POSITIVE


f1-score равен 0.3675
accuracy-score равен 0.4223
Вторая модель успешно обучена и залогирована!
🏃 View run second_model_experiment at: http://127.0.0.1:8080/#/experiments/0/runs/f363538c87124ee0a6ced8e5d6eb5376
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/0


#### Проверка результатов

In [ ]:
cfg_inference = load_config("inference_bayes_second")

client = MlflowClient()

# ===========================================================================================
# ПОЛУЧЕНИЕ МОДЕЛЕЙ
# ===========================================================================================


latest_run_model = client.search_runs(
    experiment_ids=[cfg_inference.mlflow.experiment_id],
    filter_string=f'tags.mlflow.runName = "{cfg_inference.model.run_name}"', 
    order_by=['attributes.end_time desc']
)

if not latest_run_model:
    raise ValueError(f"Не найден run: {cfg_inference.model.run_name}")

latest_run_model_id = latest_run_model[0].info.run_id
print(f"Run ID модели: {latest_run_model_id}")


vectorizer_path = f"{cfg_inference.model.artifacts_path}/{cfg_inference.model.vectorizer}"
bayes_path = f"{cfg_inference.model.artifacts_path}/{cfg_inference.model.bayes}"
encoder_path = f"{cfg_inference.model.artifacts_path}/{cfg_inference.model.encoder}"

print("Загружаем модели...")
try:
    vectorizer_file = client.download_artifacts(latest_run_model_id, vectorizer_path)
    bayes_file = client.download_artifacts(latest_run_model_id, bayes_path)
    encoder_file = client.download_artifacts(latest_run_model_id, encoder_path)
    print("✅ Модели успешно загружены")
except Exception as e:
    print(f"❌ Ошибка загрузки моделей: {e}")
    raise

# ===========================================================================================
# ПОЛУЧЕНИЕ ФУНКЦИИ ПРЕПРОЦЕССИНГА
# ===========================================================================================


latest_run_dataset = client.search_runs(
    experiment_ids=[cfg_inference.mlflow.experiment_id],
    filter_string=f'tags.mlflow.runName = "{cfg_inference.preprocess.run_name}"',
    order_by=['attributes.end_time desc']
)

if not latest_run_dataset:
    raise ValueError(f"Не найден run: {cfg_inference.preprocess.run_name}")

latest_run_dataset_id = latest_run_dataset[0].info.run_id

print("Загружаем функцию препроцессинга...")
try:
    art_loc = client.download_artifacts(latest_run_dataset_id, cfg_inference.preprocess.artifact_path)
    print("✅ Функция препроцессинга загружена")
except Exception as e:
    print(f"❌ Ошибка загрузки функции: {e}")
    raise

# ===========================================================================================
# ЗАГРУЗКА И ИСПОЛЬЗОВАНИЕ МОДЕЛЕЙ
# ===========================================================================================

print("Загружаем модели в память...")
vectorizer = joblib.load(vectorizer_file)
bayes = joblib.load(bayes_file)
encoder = joblib.load(encoder_file)

print("Загружаем функцию препроцессинга...")
with open(art_loc, 'rb') as f:
    preprocess_func = cloudpickle.load(f)

# ===========================================================================================
# ПРЕДСКАЗАНИЕ
# ===========================================================================================

# Используем текст из конфига
text = cfg_inference.test_text

print("Выполняем предсказание...")
preprocessed_text = preprocess_func(text)
done_text = vectorizer.transform([preprocessed_text])
label = bayes.predict(done_text)

print('=' * 100)
print(f'Исходный текст: "{text}"')
print(f'Предобработанный текст: "{preprocessed_text}"')
print(f'Предсказанная метка: {encoder.inverse_transform(label)[0]}')

Run ID модели: f363538c87124ee0a6ced8e5d6eb5376
Загружаем модели...


✅ Модели успешно загружены
Загружаем функцию препроцессинга...


✅ Функция препроцессинга загружена
Загружаем модели в память...


Загружаем функцию препроцессинга...
Выполняем предсказание...
Исходный текст: "Меня не привлекло такое сочетание 😭"
Предобработанный текст: "меня не привлекло такое сочетание loudly_crying_face"
Предсказанная метка: ВКУС_NEGATIVE


### Третий датасет

In [ ]:
cfg = load_config("naive_bayes_third")

with mlflow.start_run(run_name='third_model_experiment'):

    mlflow.set_tag('NaiveBayes', cfg.model.version)

    # =====================================================================================================================================
    #                                         СЧИТЫВАНИЕ ДАТАСЕТА
    # =====================================================================================================================================

    client = MlflowClient()

    dataset_runs = client.search_runs(
        experiment_ids=[cfg.mlflow.experiment_id],
        filter_string=f"tags.mlflow.runName = '{cfg.data.source_run}'",
        order_by=['attributes.end_time desc']
    )

    if not dataset_runs:
        raise ValueError(f"Не найден run: {cfg.data.source_run}")

    dataset_run = dataset_runs[0]
    dataset_run_id = dataset_run.info.run_id
    print(f"✅ Найден run с датасетом: {dataset_run_id}")


    try:
        dataframe_path = client.download_artifacts(dataset_run_id, f"datasets/{cfg.data.dataset_file}")
        df = pd.read_csv(dataframe_path)
        print("✅ Датасет загружен напрямую")
    except Exception as e:
        print(f"❌ Ошибка загрузки: {e}")
        alternative_paths = [
            cfg.data.dataset_file,
            f"artifacts/datasets/{cfg.data.dataset_file}",
            "third_experiment_dataset.csv"
        ]
        
        for path in alternative_paths:
            try:
                dataframe_path = client.download_artifacts(dataset_run_id, path)
                df = pd.read_csv(dataframe_path)
                print(f"✅ Датасет загружен: {path}")
                break
            except:
                continue
        else:
            raise ValueError("Не удалось загрузить датасет")

    display(df)

    # =====================================================================================================================================
    #                                              ВЕКТОРИЗАЦИЯ ТЕКСТА
    # =====================================================================================================================================

    vectorizer = TfidfVectorizer(
        lowercase=cfg.vectorizer.lowercase,
        analyzer=cfg.vectorizer.analyzer,
        max_features=cfg.vectorizer.max_features,
        ngram_range=tuple(cfg.vectorizer.ngram_range),
        min_df=cfg.vectorizer.min_df,
        max_df=cfg.vectorizer.max_df
    )
    
    encoder = LabelEncoder()

    # =====================================================================================================================================
    #                                         РАЗБИЕНИЕ НА TRAIN/TEST
    # =====================================================================================================================================

    df_train, df_test = train_test_split(
        df, 
        test_size=cfg.training.test_size, 
        random_state=cfg.training.random_state, 
        stratify=df['label']
    )

    X_train = df_train['span']
    X_train = vectorizer.fit_transform(X_train)

    y_train = df_train['label']
    y_train = encoder.fit_transform(y_train)

    X_test = df_test['span']
    X_test = vectorizer.transform(X_test)

    y_test = df_test['label']
    y_test = encoder.transform(y_test)

    # =====================================================================================================================================
    #                                         ОБУЧЕНИЕ И ЛОГИРОВАНИЕ МОДЕЛИ
    # =====================================================================================================================================

    mlflow.sklearn.autolog(disable=True)

    model = MultinomialNB()

    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    f1 = f1_score(y_test, y_pred, average='weighted')
    accuracy = accuracy_score(y_test, y_pred)

    print(f'f1-score равен {f1:.4f}')
    print(f'accuracy-score равен {accuracy:.4f}')

    joblib.dump(model, cfg.model.artifacts.model)
    joblib.dump(vectorizer, cfg.model.artifacts.vectorizer)
    joblib.dump(encoder, cfg.model.artifacts.encoder)
    
    mlflow.log_artifact(cfg.model.artifacts.model, 'models')
    mlflow.log_artifact(cfg.model.artifacts.vectorizer, 'models')
    mlflow.log_artifact(cfg.model.artifacts.encoder, 'models')
    
    mlflow.log_metrics({
        'f1_score': f1,
        'accuracy': accuracy
    })

    mlflow.log_params({
        'vectorizer_max_features': cfg.vectorizer.max_features,
        'vectorizer_ngram_range': str(cfg.vectorizer.ngram_range),
        'test_size': cfg.training.test_size
    })

    print("Третья модель успешно обучена и залогирована!")

✅ Найден run с датасетом: 3f371b6b84564621b3f1f656172c4f72


✅ Датасет загружен напрямую


,span,label
0,вкус был реально отличный,ВКУС_POSITIVE
1,"Я ел эти чипсы очень долго, еще года 2 назад с...",O
2,"хотелось купить очень много, но в последнее вр...",O
3,сейчас такой вкус имеет каждая 2 пачка,O
4,вкус которых стал НАМНОГО острее обычного,ВКУС_NEGATIVE
...,...,...
4015,"с чипсами всегда стоит помнить, что две трети ...",ПАЧКА_NEGATIVE
4016,"Пластиковая крышка попалась очень тугая, еле-е...",ПАЧКА_NEGATIVE
4017,"Пластиковая крышка попалась очень тугая, еле-е...",ПАЧКА_NEGATIVE
4018,Наполнеине пачки типичное для данного производ...,ПАЧКА_NEGATIVE


f1-score равен 0.5503
accuracy-score равен 0.5945
Третья модель успешно обучена и залогирована!
🏃 View run third_model_experiment at: http://127.0.0.1:8080/#/experiments/0/runs/18e0cad0f05f43b585de0ba2b6bf7464
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/0


#### Проверка результатоы

In [ ]:

cfg_inference = load_config("inference_bayes_third")

client = MlflowClient()

# ===========================================================================================
# ПОЛУЧЕНИЕ МОДЕЛЕЙ
# ===========================================================================================


latest_run_model = client.search_runs(
    experiment_ids=[cfg_inference.mlflow.experiment_id],
    filter_string=f'tags.mlflow.runName = "{cfg_inference.model.run_name}"',
    order_by=['attributes.end_time desc']
)

if not latest_run_model:
    raise ValueError(f"Не найден run: {cfg_inference.model.run_name}")

latest_run_model_id = latest_run_model[0].info.run_id
print(f"Run ID модели: {latest_run_model_id}")


vectorizer_path = f"{cfg_inference.model.artifacts_path}/{cfg_inference.model.vectorizer}"
bayes_path = f"{cfg_inference.model.artifacts_path}/{cfg_inference.model.bayes}"
encoder_path = f"{cfg_inference.model.artifacts_path}/{cfg_inference.model.encoder}"

print("Загружаем модели...")
try:
    vectorizer_file = client.download_artifacts(latest_run_model_id, vectorizer_path)
    bayes_file = client.download_artifacts(latest_run_model_id, bayes_path)
    encoder_file = client.download_artifacts(latest_run_model_id, encoder_path)
    print("✅ Модели успешно загружены")
except Exception as e:
    print(f"❌ Ошибка загрузки моделей: {e}")
    raise


# ===========================================================================================
# ЗАГРУЗКА И ИСПОЛЬЗОВАНИЕ МОДЕЛЕЙ
# ===========================================================================================

print("Загружаем модели в память...")
vectorizer = joblib.load(vectorizer_file)
bayes = joblib.load(bayes_file)
encoder = joblib.load(encoder_file)

# ===========================================================================================
# ПРЕДСКАЗАНИЕ
# ===========================================================================================

# Используем текст из конфига
text = cfg_inference.test_text

print("Выполняем предсказание...")
done_text = vectorizer.transform([text])
label = bayes.predict(done_text)

print('=' * 100)
print(f'Исходный текст: "{text}"')
print(f'Предобработанный текст: "{preprocessed_text}"')
print(f'Предсказанная метка: {encoder.inverse_transform(label)[0]}')
print('=' * 100)

Run ID модели: 18e0cad0f05f43b585de0ba2b6bf7464
Загружаем модели...


✅ Модели успешно загружены
Создаем функцию препроцессинга...
✅ Создана простая функция препроцессинга
Загружаем модели в память...
Выполняем предсказание...
Исходный текст: "Меня привлекло такое сочетание"
Предобработанный текст: "меня привлекло такое сочетание"
Предсказанная метка: ПАЧКА_POSITIVE


### Итог

In [52]:
results = pd.DataFrame({
    'Версия': ['Версия 1', 'Версия 2', 'Версия 3'],
    'F1-Score': [0.363707, 0.344306, 0.344139],
    'Accuracy': [0.413043, 0.391304, 0.391304],
    'Описание': ['Лемматизация, удаление знаков пунктуации', 'Очистка от знаков препинания и emoji to text', 'Никакой очистки']
})

results

,Версия,F1-Score,Accuracy,Описание
0,Версия 1,0.363707,0.413043,"Лемматизация, удаление знаков пунктуации"
1,Версия 2,0.344306,0.391304,Очистка от знаков препинания и emoji to text
2,Версия 3,0.344139,0.391304,Никакой очистки


## Второй эксперимент (LSTM)

### Первый датасет

In [ ]:

cfg = load_config("neural_network_first")

with mlflow.start_run(run_name='first_experiment_neural_network'):
    
    mlflow.set_tag('LSTM', cfg.model.version)

    # =====================================================================================================================================
    #                                         СЧИТЫВАНИЕ ДАТАСЕТА
    # =====================================================================================================================================

    client = MlflowClient()

    dataset_runs = client.search_runs(
        experiment_ids=[cfg.mlflow.experiment_id],
        filter_string=f"tags.mlflow.runName = '{cfg.data.source_run}'",
        order_by=['attributes.end_time desc']
    )

    if not dataset_runs:
        raise ValueError(f"Не найден run: {cfg.data.source_run}")

    dataset_run = dataset_runs[0]
    dataset_run_id = dataset_run.info.run_id
    print(f"✅ Найден run с датасетом: {dataset_run_id}")

    try:
        dataframe_path = client.download_artifacts(dataset_run_id, f"{cfg.data.dataset_path}/{cfg.data.dataset_file}")
        df = pd.read_csv(dataframe_path)
        print("✅ Датасет загружен напрямую")
    except Exception as e:
        print(f"❌ Ошибка загрузки: {e}")
        try:
            files = client.list_artifacts(dataset_run_id, cfg.data.dataset_path)
            dataframe = files[0].path
            dataframe_path = client.download_artifacts(dataset_run_id, dataframe)
            df = pd.read_csv(dataframe_path)
            print("✅ Датасет загружен через list_artifacts")
        except Exception as e2:
            print(f"❌ Ошибка загрузки через list_artifacts: {e2}")
            alternative_paths = [
                cfg.data.dataset_file,
                f"artifacts/{cfg.data.dataset_path}/{cfg.data.dataset_file}",
                "First_version.csv"
            ]
            
            for path in alternative_paths:
                try:
                    dataframe_path = client.download_artifacts(dataset_run_id, path)
                    df = pd.read_csv(dataframe_path)
                    print(f"✅ Датасет загружен: {path}")
                    break
                except:
                    continue
            else:
                raise ValueError("Не удалось загрузить датасет")

    display(df)
    
    # =====================================================================================================================================
    #                                              ВЕКТОРИЗАЦИЯ ТЕКСТА
    # =====================================================================================================================================

    mlflow.tensorflow.autolog()

    tokenizer = Tokenizer(
        num_words=cfg.tokenizer.num_words,
        oov_token=cfg.tokenizer.oov_token,
        filters=cfg.tokenizer.filters,
        lower=cfg.tokenizer.lower,
        split=cfg.tokenizer.split,
        char_level=cfg.tokenizer.char_level
    )

    tokenizer.fit_on_texts(df['span'])

    df_train, df_temp = train_test_split(
        df, 
        test_size=cfg.training.test_size, 
        random_state=cfg.training.random_state, 
        stratify=df['label']
    )

    df_test, df_val = train_test_split(
        df_temp, 
        test_size=cfg.training.val_size, 
        random_state=cfg.training.random_state, 
        stratify=df_temp['label']
    )

    X_train_vec = tokenizer.texts_to_sequences(df_train['span'])
    X_test_vec = tokenizer.texts_to_sequences(df_test['span'])
    X_val_vec = tokenizer.texts_to_sequences(df_val['span'])

    max_len_text = 0
    for i in df['span']:
        max_len_text = max(max_len_text, len(i.split(' ')))
    print(f"Максимальная длина текста: {max_len_text}")


    X_train_pad = pad_sequences(X_train_vec, max_len_text, padding='post', truncating='post')
    X_test_pad = pad_sequences(X_test_vec, max_len_text, padding='post', truncating='post')
    X_val_pad = pad_sequences(X_val_vec, max_len_text, padding='post', truncating='post')

    # =====================================================================================================================================
    #                                              ПОДГОТОВКА ТАРГЕТОВ
    # =====================================================================================================================================

    encoder = LabelEncoder()

    y_train = encoder.fit_transform(df_train['label'])
    y_test = encoder.transform(df_test['label'])
    y_val = encoder.transform(df_val['label'])

    # =====================================================================================================================================
    #                                              СОЗДАНИЕ МОДЕЛИ
    # =====================================================================================================================================

    model = Sequential()

    model.add(Embedding(
        input_dim=cfg.tokenizer.num_words, 
        output_dim=cfg.model.embedding_dim, 
        input_length=max_len_text
    ))

    model.add(Bidirectional(LSTM(
        cfg.model.lstm_units, 
        dropout=0.2, 
        recurrent_dropout=0.3
    )))

    model.add(Dropout(cfg.model.dropout_rate))

    model.add(Dense(cfg.model.dense_units, activation='softmax'))

    model.compile(
        optimizer='adam', 
        loss='sparse_categorical_crossentropy', 
        metrics=['accuracy']
    )

    model.fit(
        X_train_pad, 
        y_train, 
        batch_size=cfg.training.batch_size, 
        epochs=cfg.training.epochs, 
        validation_data=(X_val_pad, y_val)
    )

    test_loss, test_accuracy = model.evaluate(X_test_pad, y_test)
    print(f"Test Accuracy: {test_accuracy:.4f}")
    print(f"Test Loss: {test_loss:.4f}")

    model.summary()

    # =====================================================================================================================================
    #                                         ЛОГИРОВАНИЕ МОДЕЛИ
    # =====================================================================================================================================

    pred_proba = model.predict(X_test_pad)
    pred_class = np.argmax(pred_proba, axis=1)
    
    f1 = f1_score(pred_class, y_test, average='weighted')
    print(f'f1-score у модели равен {f1:.4f}')

    mlflow.log_metric('f1_score', f1)
    mlflow.log_metric('test_accuracy', test_accuracy)
    mlflow.log_metric('test_loss', test_loss)

    model.save("LSTM_ver_1.keras")

    with open(cfg.model.artifacts.tokenizer, 'wb') as f:
        cloudpickle.dump(tokenizer, f)
    
    with open(cfg.model.artifacts.encoder, 'wb') as f:
        cloudpickle.dump(encoder, f)

    mlflow.log_artifact(cfg.model.artifacts.model, 'models')
    mlflow.log_artifact(cfg.model.artifacts.tokenizer, 'models') 
    mlflow.log_artifact(cfg.model.artifacts.encoder, 'models')

    mlflow.log_params({
        'num_words': cfg.tokenizer.num_words,
        'embedding_dim': cfg.model.embedding_dim,
        'lstm_units': cfg.model.lstm_units,
        'dropout_rate': cfg.model.dropout_rate,
        'batch_size': cfg.training.batch_size,
        'epochs': cfg.training.epochs
    })

    print("✅ Нейронная сеть успешно обучена и залогирована!")

✅ Найден run с датасетом: d63ca4ae2b054f49b3ce30a18216827a


✅ Датасет загружен напрямую


,span,label
0,вкус реально отличный,ВКУС_POSITIVE
1,я есть чипсы очень долго год 2 назад советоват...,O
2,хотеться купить очень последний время стать оч...,O
3,вкус иметь каждый 2 пачка,O
4,вкус который стать намного острый обычный,ВКУС_NEGATIVE
...,...,...
2503,ярко красиво фиолетовый цвет любить новогодний...,ПАЧКА_POSITIVE
2504,чипсы довольно жирный не критично заметно,ТЕКСТУРА_NEUTRAL
2505,сам чипсы не идеально круглый,ТЕКСТУРА_NEUTRAL
2506,красивый золотистый нет пригорелый поломать ми...,ТЕКСТУРА_POSITIVE


Максимальная длина текста: 73


c:\Users\Smart\PycharmProjects\NLP_and_MLOPS_course_work\.venv\Lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
2025/11/25 10:35:08 WARNING mlflow.tensorflow: Encountered unexpected error while inferring batch size from training dataset: Sequential model 'sequential_2' has no defined input shape yet.


...

 Total params: 1,068,320 (4.08 MB)

 Trainable params: 356,106 (1.36 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 712,214 (2.72 MB)

8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 149ms/step
f1-score у модели равен 0.4819
✅ Нейронная сеть успешно обучена и залогирована!
🏃 View run first_experiment_neural_network at: http://127.0.0.1:8080/#/experiments/0/runs/5793fbc2bec14768b3df062c87b0ba98
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/0


#### Проверка результатов

In [ ]:
cfg_inference = load_config("inference_neural_network")

client = MlflowClient()

# ===========================================================================================
# ПОЛУЧЕНИЕ МОДЕЛЕЙ
# ===========================================================================================

latest_run_model = client.search_runs(
    experiment_ids=[cfg_inference.mlflow.experiment_id],
    filter_string=f'tags.mlflow.runName = "{cfg_inference.model.run_name}"',
    order_by=['attributes.end_time desc']
)

if not latest_run_model:
    raise ValueError(f"Не найден run: {cfg_inference.model.run_name}")

latest_run_model_id = latest_run_model[0].info.run_id
print(f"Run ID модели: {latest_run_model_id}")

model_path = f"{cfg_inference.model.artifacts_path}/{cfg_inference.model.model_file}"
tokenizer_path = f"{cfg_inference.model.artifacts_path}/{cfg_inference.model.tokenizer}"
encoder_path = f"{cfg_inference.model.artifacts_path}/{cfg_inference.model.encoder}"

print("Загружаем модели...")
try:
    model_file = client.download_artifacts(latest_run_model_id, model_path)
    tokenizer_file = client.download_artifacts(latest_run_model_id, tokenizer_path)
    encoder_file = client.download_artifacts(latest_run_model_id, encoder_path)
    print("✅ Модели успешно загружены")
except Exception as e:
    print(f"❌ Ошибка загрузки моделей: {e}")
    raise

# ===========================================================================================
# ЗАГРУЗКА И ИСПОЛЬЗОВАНИЕ МОДЕЛЕЙ
# ===========================================================================================

print("Загружаем модели в память...")
model_keras = tf.keras.models.load_model(model_file)
encoder = joblib.load(encoder_file)
tokenizer = joblib.load(tokenizer_file)

print("✅ Все модели загружены в память")

# ===========================================================================================
# ПРЕДСКАЗАНИЕ
# ===========================================================================================

text = cfg_inference.test_text

print("Выполняем предсказание...")

tokenized_text = tokenizer.texts_to_sequences([text])

max_len = model_keras.input_shape[1]
print(f"Максимальная длина последовательности: {max_len}")

padded_text = pad_sequences(tokenized_text, maxlen=max_len, padding='post', truncating='post')

pred = model_keras.predict(padded_text)

predicted_class_ind = np.argmax(pred, axis=1)
predicted_class = encoder.inverse_transform(predicted_class_ind)

print('=' * 100)
print(f'Исходный текст: "{text}"')
print(f'Предсказанная метка: {predicted_class[0]}')
print(f'Вероятности по классам:')
for i, prob in enumerate(pred[0]):
    class_name = encoder.inverse_transform([i])[0]
    print(f'  {class_name}: {prob:.4f}')
print('=' * 100)

Run ID модели: 5793fbc2bec14768b3df062c87b0ba98
Загружаем модели...


✅ Модели успешно загружены
Загружаем модели в память...
✅ Все модели загружены в память
Выполняем предсказание...
Максимальная длина последовательности: 73
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Исходный текст: "В целом, чипсы Ашан Красная птица Барбекю вполне съедобные"
Предсказанная метка: O
Вероятности по классам:
  O: 0.8583
  ВКУС_NEGATIVE: 0.0006
  ВКУС_NEUTRAL: 0.0257
  ВКУС_POSITIVE: 0.0022
  ПАЧКА_NEGATIVE: 0.0020
  ПАЧКА_NEUTRAL: 0.0415
  ПАЧКА_POSITIVE: 0.0004
  ТЕКСТУРА_NEGATIVE: 0.0356
  ТЕКСТУРА_NEUTRAL: 0.0226
  ТЕКСТУРА_POSITIVE: 0.0112


### Второй датасет

In [ ]:

cfg = load_config("neural_network_second")

with mlflow.start_run(run_name='second_experiment_neural_network'):
    
    mlflow.set_tag('CNN', cfg.model.version)

    # =====================================================================================================================================
    #                                         СЧИТЫВАНИЕ ДАТАСЕТА
    # =====================================================================================================================================

    client = MlflowClient()

    dataset_runs = client.search_runs(
        experiment_ids=[cfg.mlflow.experiment_id],
        filter_string=f"tags.mlflow.runName = '{cfg.data.source_run}'",
        order_by=['attributes.end_time desc']
    )

    if not dataset_runs:
        raise ValueError(f"Не найден run: {cfg.data.source_run}")

    dataset_run = dataset_runs[0]
    dataset_run_id = dataset_run.info.run_id
    print(f"✅ Найден run с датасетом: {dataset_run_id}")

    try:
        dataset_path = f"{cfg.data.dataset_path}/{cfg.data.dataset_file}"
        art = client.download_artifacts(dataset_run_id, dataset_path)
        df = pd.read_csv(art)
        print(f"✅ Датасет загружен: {dataset_path}")
    except Exception as e:
        print(f"❌ Ошибка загрузки: {e}")
        alternative_paths = [
            cfg.data.dataset_file,
            f"artifacts/{cfg.data.dataset_path}/{cfg.data.dataset_file}",
            "Second_version.csv"
        ]
        
        for path in alternative_paths:
            try:
                art = client.download_artifacts(dataset_run_id, path)
                df = pd.read_csv(art)
                print(f"✅ Датасет загружен: {path}")
                break
            except:
                continue
        else:
            raise ValueError("Не удалось загрузить датасет")

    display(df)

    # =====================================================================================================================================
    #                                         ТОКЕНИЗАЦИЯ ТЕКСТА
    # =====================================================================================================================================

    mlflow.tensorflow.autolog()

    tokenizer_config = {k: v for k, v in cfg.tokenizer.items() if v is not None}
    tokenizer = Tokenizer(**tokenizer_config)
    
    tokenizer.fit_on_texts(df['span'])

    df_train, df_temp = train_test_split(
        df, 
        test_size=cfg.training.test_size, 
        random_state=cfg.training.random_state
    )
    
    df_test, df_val = train_test_split(
        df_temp, 
        test_size=cfg.training.val_size, 
        random_state=cfg.training.random_state
    )

    X_train_vec = tokenizer.texts_to_sequences(df_train['span'])
    X_test_vec = tokenizer.texts_to_sequences(df_test['span'])
    X_val_vec = tokenizer.texts_to_sequences(df_val['span'])

    X_train_pad = pad_sequences(X_train_vec, maxlen=cfg.training.max_sequence_length, padding='post', truncating='post')
    X_test_pad = pad_sequences(X_test_vec, maxlen=cfg.training.max_sequence_length, padding='post', truncating='post')
    X_val_pad = pad_sequences(X_val_vec, maxlen=cfg.training.max_sequence_length, padding='post', truncating='post')

    # =====================================================================================================================================
    #                                         ПОДГОТОВКА ТАРГЕТОВ
    # =====================================================================================================================================

    encoder = LabelEncoder()

    y_train = encoder.fit_transform(df_train['label'])
    y_test = encoder.transform(df_test['label'])
    y_val = encoder.transform(df_val['label'])

    # =====================================================================================================================================
    #                                         СОЗДАНИЕ МОДЕЛИ CNN
    # =====================================================================================================================================

    model = Sequential()

    # Embedding слой
    model.add(Embedding(
        input_dim=cfg.tokenizer.num_words, 
        output_dim=cfg.model.embedding_dim, 
        input_length=cfg.training.max_sequence_length
    ))

    # Conv1D слой
    model.add(Conv1D(
        filters=cfg.model.conv_filters,
        kernel_size=cfg.model.conv_kernel_size,
        activation=cfg.model.conv_activation
    ))

    model.add(Dropout(cfg.model.dropout_rate_1))

    model.add(GlobalMaxPooling1D())

    model.add(Dense(cfg.model.dense_units_1, activation=cfg.model.dense_activation_1))

    model.add(Dropout(cfg.model.dropout_rate_2))

    model.add(Dense(cfg.model.dense_units_2, activation=cfg.model.dense_activation_2))

    model.compile(
        loss="sparse_categorical_crossentropy", 
        optimizer='adam', 
        metrics=['accuracy']
    )

    model.fit(
        X_train_pad, 
        y_train, 
        batch_size=cfg.training.batch_size, 
        epochs=cfg.training.epochs, 
        validation_data=(X_val_pad, y_val)
    )

    model.summary()

    test_loss, test_accuracy = model.evaluate(X_test_pad, y_test)
    print(f"Test Accuracy: {test_accuracy:.4f}")
    print(f"Test Loss: {test_loss:.4f}")

    # =====================================================================================================================================
    #                                         ЛОГИРОВАНИЕ МОДЕЛИ
    # =====================================================================================================================================

    pred = model.predict(X_test_pad)
    pred_class = np.argmax(pred, axis=1)

    f1 = f1_score(pred_class, y_test, average='weighted')
    print(f'f1-score у модели равен {f1:.4f}')

    mlflow.log_metric('f1_score', f1)
    mlflow.log_metric('test_accuracy', test_accuracy)
    mlflow.log_metric('test_loss', test_loss)

    model.save(cfg.model.artifacts.model)

    joblib.dump(tokenizer, cfg.model.artifacts.tokenizer)
    joblib.dump(encoder, cfg.model.artifacts.encoder)

    mlflow.log_artifact(cfg.model.artifacts.model, 'models')
    mlflow.log_artifact(cfg.model.artifacts.tokenizer, 'models')
    mlflow.log_artifact(cfg.model.artifacts.encoder, 'models')

    mlflow.log_params({
        'num_words': cfg.tokenizer.num_words,
        'embedding_dim': cfg.model.embedding_dim,
        'conv_filters': cfg.model.conv_filters,
        'conv_kernel_size': cfg.model.conv_kernel_size,
        'dropout_rate_1': cfg.model.dropout_rate_1,
        'dropout_rate_2': cfg.model.dropout_rate_2,
        'dense_units_1': cfg.model.dense_units_1,
        'batch_size': cfg.training.batch_size,
        'epochs': cfg.training.epochs,
        'max_sequence_length': cfg.training.max_sequence_length
    })

    print("✅ CNN модель успешно обучена и залогирована!")

✅ Найден run с датасетом: 38adc29950984b408e94b3a3b5c0cb6c


✅ Датасет загружен: datasets/second_experiment_dataset.csv


,span,label
0,вкус был реально отличный,ВКУС_POSITIVE
1,я ел эти чипсы очень долго еще года 2 назад со...,O
2,хотелось купить очень много но в последнее вре...,O
3,сейчас такой вкус имеет каждая 2 пачка,O
4,вкус которых стал намного острее обычного,ВКУС_NEGATIVE
...,...,...
2503,ярко красиво фиолетовый цвет я люблю новогодне...,ПАЧКА_POSITIVE
2504,чипсы довольно жирные не критично но заметно,ТЕКСТУРА_NEUTRAL
2505,сами чипсы не идеально круглые,ТЕКСТУРА_NEUTRAL
2506,красивые золотистые нет пригорелых поломанных ...,ТЕКСТУРА_POSITIVE


c:\Users\Smart\PycharmProjects\NLP_and_MLOPS_course_work\.venv\Lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
2025/11/25 10:38:10 WARNING mlflow.tensorflow: Encountered unexpected error while inferring batch size from training dataset: Sequential model 'sequential_3' has no defined input shape yet.


Epoch 1/40
126/126 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.1550 - loss: 2.2458 - val_accuracy: 0.1753 - val_loss: 2.2042
Epoch 2/40
126/126 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.1780 - loss: 2.1852 - val_accuracy: 0.1753 - val_loss: 2.1808
Epoch 3/40
126/126 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.2278 - loss: 2.0850 - val_accuracy: 0.2032 - val_loss: 2.0804
Epoch 4/40
126/126 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.3255 - loss: 1.8641 - val_accuracy: 0.3147 - val_loss: 1.9178
Epoch 5/40
126/126 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.4556 - loss: 1.5206 - val_accuracy: 0.3705 - val_loss: 1.7730
Epoch 6/40
126/126 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.5259 - loss: 1.3181 - val_accuracy: 0.4263 - val_loss: 1.7265
Epoch 7/40
126/126 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.5877 - loss: 1.1221 - val_accuracy: 0.4223 - val_loss: 1.6844
Epoch 8/40
126/126 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.6251 - loss: 0.9918 - val_accuracy: 0

...

 Total params: 404,624 (1.54 MB)

 Trainable params: 134,874 (526.85 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 269,750 (1.03 MB)

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.4143 - loss: 3.4621
Test Accuracy: 0.4143
Test Loss: 3.4621
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
f1-score у модели равен 0.4187
✅ CNN модель успешно обучена и залогирована!
🏃 View run second_experiment_neural_network at: http://127.0.0.1:8080/#/experiments/0/runs/cd028d31ab9247ac8198400dfb0f1347
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/0


#### Проверка результатов

In [ ]:
cfg_inference = load_config("inference_neural_network_second")

client = MlflowClient()

# ===========================================================================================
# ПОЛУЧЕНИЕ МОДЕЛЕЙ
# ===========================================================================================

latest_run_model = client.search_runs(
    experiment_ids=[cfg_inference.mlflow.experiment_id],
    filter_string=f'tags.mlflow.runName = "{cfg_inference.model.run_name}"',
    order_by=['attributes.end_time desc']
)

if not latest_run_model:
    raise ValueError(f"Не найден run: {cfg_inference.model.run_name}")

latest_run_model_id = latest_run_model[0].info.run_id
print(f"Run ID модели: {latest_run_model_id}")

model_path = f"{cfg_inference.model.artifacts_path}/{cfg_inference.model.model_file}"
tokenizer_path = f"{cfg_inference.model.artifacts_path}/{cfg_inference.model.tokenizer}"
encoder_path = f"{cfg_inference.model.artifacts_path}/{cfg_inference.model.encoder}"

print("Загружаем модели...")
try:
    model_file = client.download_artifacts(latest_run_model_id, model_path)
    tokenizer_file = client.download_artifacts(latest_run_model_id, tokenizer_path)
    encoder_file = client.download_artifacts(latest_run_model_id, encoder_path)
    print("✅ Модели успешно загружены")
except Exception as e:
    print(f"❌ Ошибка загрузки моделей: {e}")
    raise

# ===========================================================================================
# ЗАГРУЗКА И ИСПОЛЬЗОВАНИЕ МОДЕЛЕЙ
# ===========================================================================================

print("Загружаем модели в память...")
model_keras = tf.keras.models.load_model(model_file)
encoder = joblib.load(encoder_file)
tokenizer = joblib.load(tokenizer_file)

print("✅ Все модели загружены в память")

# ===========================================================================================
# ПРЕДСКАЗАНИЕ
# ===========================================================================================


text = cfg_inference.test_text

print("Выполняем предсказание...")


tokenized_text = tokenizer.texts_to_sequences([text])


max_len = model_keras.input_shape[1]
print(f"Максимальная длина последовательности: {max_len}")


padded_text = pad_sequences(tokenized_text, maxlen=max_len, padding='post', truncating='post')


pred = model_keras.predict(padded_text)


predicted_class_ind = np.argmax(pred, axis=1)
predicted_class = encoder.inverse_transform(predicted_class_ind)

print('=' * 100)
print(f'Исходный текст: "{text}"')
print(f'Предсказанная метка: {predicted_class[0]}')
print(f'Вероятности по классам:')
for i, prob in enumerate(pred[0]):
    class_name = encoder.inverse_transform([i])[0]
    print(f'  {class_name}: {prob:.4f}')
print('=' * 100)

Run ID модели: cd028d31ab9247ac8198400dfb0f1347
Загружаем модели...


✅ Модели успешно загружены
Загружаем модели в память...
✅ Все модели загружены в память
Выполняем предсказание...
Максимальная длина последовательности: 100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 136ms/step
Исходный текст: "В целом, чипсы Ашан Красная птица Барбекю вполне съедобные"
Предсказанная метка: ВКУС_NEUTRAL
Вероятности по классам:
  O: 0.4616
  ВКУС_NEGATIVE: 0.0000
  ВКУС_NEUTRAL: 0.5211
  ВКУС_POSITIVE: 0.0021
  ПАЧКА_NEGATIVE: 0.0015
  ПАЧКА_NEUTRAL: 0.0137
  ПАЧКА_POSITIVE: 0.0000
  ТЕКСТУРА_NEGATIVE: 0.0000
  ТЕКСТУРА_NEUTRAL: 0.0000
  ТЕКСТУРА_POSITIVE: 0.0001


## Третий эксперимент (Трансформеры)

### Первый датасет

In [ ]:
cfg = load_config("transformers_first")

with mlflow.start_run(run_name='transformers_experiment_1'):
    
    mlflow.set_tag('Transformers', cfg.model.version)
    client = MlflowClient()

    # =====================================================================================================================================
    #                                         СЧИТЫВАНИЕ ДАТАСЕТА
    # =====================================================================================================================================
    

    dataset_runs = client.search_runs(
        experiment_ids=[cfg.mlflow.experiment_id],
        filter_string=f"tags.mlflow.runName = '{cfg.data.source_run}'",
        order_by=['attributes.end_time desc']
    )

    if not dataset_runs:
        raise ValueError(f"Не найден run: {cfg.data.source_run}")

    dataset_run = dataset_runs[0]
    dataset_run_id = dataset_run.info.run_id
    print(f"✅ Найден run с датасетом: {dataset_run_id}")


    try:
        dataset_path = f"{cfg.data.dataset_path}/{cfg.data.dataset_file}"
        art_loc = client.download_artifacts(dataset_run_id, dataset_path)
        df = pd.read_csv(art_loc)
        print(f"✅ Датасет загружен: {dataset_path}")
    except Exception as e:
        print(f"❌ Ошибка загрузки: {e}")

        alternative_paths = [
            cfg.data.dataset_file,
            f"artifacts/{cfg.data.dataset_path}/{cfg.data.dataset_file}",
            "First_version.csv"
        ]
        
        for path in alternative_paths:
            try:
                art_loc = client.download_artifacts(dataset_run_id, path)
                df = pd.read_csv(art_loc)
                print(f"✅ Датасет загружен: {path}")
                break
            except:
                continue
        else:
            raise ValueError("Не удалось загрузить датасет")


    df = df[[cfg.data.text_column, cfg.data.label_column]]
    df = df.rename(columns={cfg.data.label_column: "labels"})
    display(df)

    # =====================================================================================================================================
    #                                         ПОДГОТОВКА ДАТАСЕТА/ТАРГЕТОВ
    # =====================================================================================================================================
    
    df_train, df_temp = train_test_split(
        df, 
        test_size=cfg.training.test_size, 
        random_state=cfg.training.random_state, 
        stratify=df['labels']
    )
    
    df_test, df_val = train_test_split(
        df_temp, 
        test_size=cfg.training.val_size, 
        random_state=cfg.training.random_state, 
        stratify=df_temp['labels']
    )

    encoder = LabelEncoder()
    y_train = encoder.fit_transform(df_train['labels'])
    y_test = encoder.transform(df_test['labels'])
    y_val = encoder.transform(df_val['labels'])

    # =====================================================================================================================================
    #                                         ПОДГОТОВКА МОДЕЛИ
    # =====================================================================================================================================
    
    mlflow.transformers.autolog()


    tokenizer = AutoTokenizer.from_pretrained(cfg.model.model_name)

    model = AutoModelForSequenceClassification.from_pretrained(
        cfg.model.model_name, 
        num_labels=len(encoder.classes_), 
        id2label={i: label for i, label in enumerate(encoder.classes_)},
        label2id={label: i for i, label in enumerate(encoder.classes_)}
    )
    

    dataset_train = Dataset.from_pandas(df_train.assign(labels=y_train))    
    dataset_test = Dataset.from_pandas(df_test.assign(labels=y_test))    
    dataset_val = Dataset.from_pandas(df_val.assign(labels=y_val))


    def tokenize_dataset(row):
        tokenizer_config = {k: v for k, v in cfg.tokenizer.items() if v is not None}
        return tokenizer(row[cfg.data.text_column], **tokenizer_config)

    dataset_tokenized_train = dataset_train.map(tokenize_dataset, batched=False)
    dataset_tokenized_test = dataset_test.map(tokenize_dataset, batched=False)
    dataset_tokenized_val = dataset_val.map(tokenize_dataset, batched=False)


    data_collator = DataCollatorWithPadding(
        tokenizer=tokenizer,
        padding=True,
        return_tensors='pt'
    )
    

    training_args = TrainingArguments(
        output_dir=cfg.training.output_dir,
        overwrite_output_dir=True,
        logging_dir='./logs/',
        num_train_epochs=cfg.training.num_train_epochs,
        learning_rate=cfg.training.learning_rate,
        per_device_train_batch_size=cfg.training.per_device_train_batch_size,
        per_device_eval_batch_size=cfg.training.per_device_eval_batch_size,
        eval_strategy=cfg.training.eval_strategy,
        save_strategy=cfg.training.save_strategy,
        warmup_ratio=cfg.training.warmup_ratio,
        lr_scheduler_type=cfg.training.lr_scheduler_type,
        metric_for_best_model=cfg.training.metric_for_best_model,
        weight_decay=cfg.training.weight_decay,
        load_best_model_at_end=cfg.training.load_best_model_at_end,
        save_total_limit=cfg.training.save_total_limit,
        max_grad_norm=cfg.training.max_grad_norm,
        logging_steps=cfg.training.logging_steps
    )


    def compute_metrics(eval_pred):
        predictions, labels = eval_pred
        predictions = np.argmax(predictions, axis=1)
        
        return {
            'f1-score': f1_score(labels, predictions, average='weighted'),
            'accuracy': accuracy_score(labels, predictions)
        }


    trainer = Trainer(
        model=model, 
        args=training_args, 
        train_dataset=dataset_tokenized_train,
        eval_dataset=dataset_tokenized_val,
        compute_metrics=compute_metrics,
        data_collator=data_collator,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=cfg.training.early_stopping_patience)]
    )


    trainer.train()


    final_metrics = trainer.evaluate(dataset_tokenized_test)
    print(f"✅ Final test metrics: {final_metrics}")

    # =====================================================================================================================================
    #                                         СОХРАНЕНИЕ И ЛОГИРОВАНИЕ МОДЕЛИ
    # =====================================================================================================================================
       

    model_dir = cfg.model.artifacts.model_dir
    tokenizer_dir = cfg.model.artifacts.tokenizer_dir
    
    os.makedirs(model_dir, exist_ok=True)
    os.makedirs(tokenizer_dir, exist_ok=True)
    
    model.save_pretrained(model_dir)
    tokenizer.save_pretrained(tokenizer_dir)


    mlflow.log_artifacts(model_dir, "model")
    mlflow.log_artifacts(tokenizer_dir, "tokenizer")


    mlflow.log_params({
        'model_name': cfg.model.model_name,
        'num_labels': len(encoder.classes_),
        'num_train_epochs': cfg.training.num_train_epochs,
        'learning_rate': cfg.training.learning_rate,
        'batch_size': cfg.training.per_device_train_batch_size,
        'early_stopping_patience': cfg.training.early_stopping_patience
    })

    print("✅ Трансформер успешно обучен и залогирован!")

✅ Найден run с датасетом: d63ca4ae2b054f49b3ce30a18216827a


✅ Датасет загружен: datasets/first_experiment_dataset.csv


,span,labels
0,вкус реально отличный,ВКУС_POSITIVE
1,я есть чипсы очень долго год 2 назад советоват...,O
2,хотеться купить очень последний время стать оч...,O
3,вкус иметь каждый 2 пачка,O
4,вкус который стать намного острый обычный,ВКУС_NEGATIVE
...,...,...
2503,ярко красиво фиолетовый цвет любить новогодний...,ПАЧКА_POSITIVE
2504,чипсы довольно жирный не критично заметно,ТЕКСТУРА_NEUTRAL
2505,сам чипсы не идеально круглый,ТЕКСТУРА_NEUTRAL
2506,красивый золотистый нет пригорелый поломать ми...,ТЕКСТУРА_POSITIVE


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Map: 100%|██████████| 251/251 [00:00<00:00, 2480.36 examples/s]
c:\Users\Smart\PycharmProjects\NLP_and_MLOPS_course_work\.venv\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss,F1-score,Accuracy
1,1.804900,1.766063,0.316007,0.382470


c:\Users\Smart\PycharmProjects\NLP_and_MLOPS_course_work\.venv\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


🏃 View run transformers_experiment_1 at: http://127.0.0.1:8080/#/experiments/0/runs/ee701fc4d7254031889b2633b9423bec
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/0


KeyboardInterrupt: 

#### Проверка результата

In [ ]:
cfg_inference = load_config("inference_transformers_first")

client = MlflowClient()

# ===========================================================================================
# ПОЛУЧЕНИЕ МОДЕЛЕЙ
# ===========================================================================================


latest_run_model = client.search_runs(
    experiment_ids=[cfg_inference.mlflow.experiment_id],
    filter_string=f'tags.mlflow.runName = "{cfg_inference.model.run_name}"',
    order_by=['attributes.end_time desc']
)

if not latest_run_model:
    raise ValueError(f"Не найден run: {cfg_inference.model.run_name}")

latest_run_model_id = latest_run_model[0].info.run_id
print(f"Run ID модели: {latest_run_model_id}")


print("Загружаем модель и токенизатор...")
try:

    model_dir = client.download_artifacts(latest_run_model_id, cfg_inference.model.artifacts_path)
    tokenizer_dir = client.download_artifacts(latest_run_model_id, cfg_inference.model.tokenizer_path)
    print("✅ Модель и токенизатор успешно загружены")
except Exception as e:
    print(f"❌ Ошибка загрузки моделей: {e}")
    raise

# ===========================================================================================
# ЗАГРУЗКА И ИСПОЛЬЗОВАНИЕ МОДЕЛЕЙ
# ===========================================================================================

print("Загружаем модели в память...")
model = AutoModelForSequenceClassification.from_pretrained(model_dir)
tokenizer = AutoTokenizer.from_pretrained(tokenizer_dir)

print("✅ Модель и токенизатор загружены в память")

# ===========================================================================================
# ПРЕДСКАЗАНИЕ
# ===========================================================================================


text = cfg_inference.test_text

print("Выполняем предсказание...")


tokenizer_config = {k: v for k, v in cfg_inference.tokenizer.items() if v is not None}
inputs = tokenizer(text, **tokenizer_config)


with torch.no_grad():
    outputs = model(**inputs)
    predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)

predicted_class_idx = predictions.argmax().item()
predicted_prob = predictions.max().item()

id2label = model.config.id2label
predicted_label = id2label[predicted_class_idx]

print('=' * 100)
print(f'Исходный текст: "{text}"')
print(f'Предсказанная метка: {predicted_label}')
print(f'Вероятность: {predicted_prob:.4f}')
print(f'Все вероятности:')
for i, prob in enumerate(predictions[0]):
    label = id2label[i]
    print(f'  {label}: {prob:.4f}')
print('=' * 100)

Run ID модели: fa6c6a5c7c5f45408cc33664a511cadb
Загружаем модель и токенизатор...


✅ Модель и токенизатор успешно загружены
Загружаем модели в память...
✅ Модель и токенизатор загружены в память
Выполняем предсказание...
Исходный текст: "В целом, чипсы Ашан Красная птица Барбекю вполне съедобные"
Предсказанная метка: ВКУС_NEUTRAL
Вероятность: 0.2972
Все вероятности:
  O: 0.1258
  ВКУС_NEGATIVE: 0.1413
  ВКУС_NEUTRAL: 0.2972
  ВКУС_POSITIVE: 0.2595
  ПАЧКА_NEGATIVE: 0.0235
  ПАЧКА_NEUTRAL: 0.0211
  ПАЧКА_POSITIVE: 0.0271
  ТЕКСТУРА_NEGATIVE: 0.0261
  ТЕКСТУРА_NEUTRAL: 0.0381
  ТЕКСТУРА_POSITIVE: 0.0404


### Третий датасет

In [ ]:
cfg = load_config("transformer_third")


with mlflow.start_run(run_name=cfg.mlflow.run_name):

    mlflow.set_tag("Dataset_version", cfg.mlflow.dataset_version)
    mlflow.log_param("model_name", cfg.model.name)

    # ---------------------------
    #   Загружаем датасет
    # ---------------------------

    dataset_runs = client.search_runs(
        experiment_ids=[cfg.mlflow.experiment_name], 
        filter_string=f"tags.mlflow.runName = 'Third dataset'",
        order_by=['attributes.end_time desc']
    )

    if not dataset_runs:
        raise ValueError("Не найден run с датасетом 'Third dataset'")

    dataset_run_id = dataset_runs[0].info.run_id
    print(f"✅ Найден run с датасетом: {dataset_run_id}")


    dataset_path_in_run = "datasets/third_experiment_dataset.csv" 
    artifact_local_path = client.download_artifacts(dataset_run_id, dataset_path_in_run)


    df = pd.read_csv(artifact_local_path)
    print(f"✅ Датасет загружен из run: {artifact_local_path}")

    encoder = LabelEncoder()
    df["labels"] = encoder.fit_transform(df["label"])

    mlflow.log_param("num_labels", len(encoder.classes_))

    # ---------------------------
    #   Трейн/Вал/Тест
    # ---------------------------
    df_train, df_temp = train_test_split(
        df,
        test_size=1 - cfg.data.train_size,
        random_state=cfg.data.random_state,
        shuffle=True,
        stratify=df["labels"]
    )

    df_val, df_test = train_test_split(
        df_temp,
        test_size=cfg.data.val_size,
        random_state=cfg.data.random_state,
        shuffle=True,
        stratify=df_temp["labels"]
    )

    # ---------------------------
    #   HuggingFace dataset
    # ---------------------------
    dataset_train = Dataset.from_pandas(df_train)
    dataset_val = Dataset.from_pandas(df_val)
    dataset_test = Dataset.from_pandas(df_test)

    # ---------------------------
    #   Токенизация
    # ---------------------------
    tokenizer = AutoTokenizer.from_pretrained(cfg.model.name)

    def tokenize(batch):
        return tokenizer(
            batch["span"],
            truncation=True,
            max_length=cfg.model.max_length
        )

    tokenized_train = dataset_train.map(tokenize, batched=True)
    tokenized_val = dataset_val.map(tokenize, batched=True)
    tokenized_test = dataset_test.map(tokenize, batched=True)

    data_collator = DataCollatorWithPadding(
        tokenizer=tokenizer,
        padding=True,
        max_length=cfg.model.max_length,
        return_tensors="pt"
    )

    # ---------------------------
    #   Инициализация модели
    # ---------------------------
    num_labels = len(encoder.classes_)

    model = AutoModelForSequenceClassification.from_pretrained(
        cfg.model.name,
        num_labels=num_labels,
        id2label={i: label for i, label in enumerate(encoder.classes_)},
        label2id={label: i for i, label in enumerate(encoder.classes_)},
        hidden_dropout_prob=cfg.model.dropout.hidden,
        attention_probs_dropout_prob=cfg.model.dropout.attention,
        classifier_dropout=cfg.model.dropout.classifier
    )

    # ---------------------------
    #   Training Arguments
    # ---------------------------
    training_args = TrainingArguments(
        output_dir=cfg.training.output_dir,
        num_train_epochs=cfg.training.num_train_epochs,
        learning_rate=cfg.training.learning_rate,
        per_device_train_batch_size=cfg.training.per_device_train_batch_size,
        per_device_eval_batch_size=cfg.training.per_device_eval_batch_size,
        warmup_ratio=cfg.training.warmup_ratio,
        lr_scheduler_type=cfg.training.lr_scheduler_type,
        evaluation_strategy=cfg.training.eval_strategy,
        save_strategy=cfg.training.save_strategy,
        weight_decay=cfg.training.weight_decay,
        logging_steps=cfg.training.logging_steps,
        save_total_limit=cfg.training.save_total_limit,
        max_grad_norm=cfg.training.max_grad_norm,
        load_best_model_at_end=True,
        metric_for_best_model="f1-score",
        dataloader_num_workers=2
    )

    # ---------------------------
    #   Метрики
    # ---------------------------
    def compute_metrics(eval_pred):
        logits, labels = eval_pred
        preds = np.argmax(logits, axis=1)

        return {
            "f1-score": f1_score(labels, preds, average="weighted"),
            "accuracy": accuracy_score(labels, preds)
        }

    # ---------------------------
    #   Обучение
    # ---------------------------
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_train,
        eval_dataset=tokenized_val,
        compute_metrics=compute_metrics,
        data_collator=data_collator,
        tokenizer=tokenizer,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=cfg.training.early_stopping_patience)]
    )

    trainer.train()

    # ---------------------------
    #   Оценка
    # ---------------------------
    eval_test = trainer.evaluate(tokenized_test)
    mlflow.log_metrics(eval_test)

    # ---------------------------
    #   Сохранение модели
    # ---------------------------
    trainer.save_model("best_transformer_model")
    tokenizer.save_pretrained("best_transformer_tokenizer")

    mlflow.log_artifact("best_transformer_model", artifact_path="models")
    mlflow.log_artifact("best_transformer_tokenizer", artifact_path="tokenizer")

    print("Модель обучена и сохранена!")

#### Проверка результатов (инференс)

In [ ]:
cfg = load_config("inference_transformers_third")

client = MlflowClient(tracking_uri=cfg.mlflow.tracking_uri)

# ================================
# 2. Ищем нужный run в MLflow
# ================================
print("Ищем run с моделью...")

runs = client.search_runs(
    experiment_ids=[cfg.mlflow.experiment_id],
    filter_string=f'tags.mlflow.runName = "{cfg.model.run_name}"',
    order_by=["attributes.end_time desc"]
)

if not runs:
    raise ValueError(f"❌ Run '{cfg.model.run_name}' не найден!")

run_id = runs[0].info.run_id
print(f"✅ Найден run: {run_id}")

# ================================
# 3. Скачиваем артефакты
# ================================
print("Скачиваем модель и токенизатор...")

model_dir = client.download_artifacts(run_id, cfg.model.artifacts_path)
tokenizer_dir = client.download_artifacts(run_id, cfg.model.tokenizer_path)

print(f"📁 Модель скачана: {model_dir}")
print(f"📁 Токенизатор скачан: {tokenizer_dir}")

# ================================
# 4. Загружаем модель в память
# ================================
print("Загружаем модель...")

model = AutoModelForSequenceClassification.from_pretrained(model_dir)
tokenizer = AutoTokenizer.from_pretrained(tokenizer_dir)

id2label = model.config.id2label

print("✅ Модель загружена")

# ================================
# 5. Предсказание
# ================================
text = cfg.test_text
print("Тестовый текст:", text)

tokenizer_cfg = {k: v for k, v in cfg.tokenizer.items() if v is not None}

inputs = tokenizer(text, **tokenizer_cfg)

with torch.no_grad():
    outputs = model(**inputs)
    probs = torch.nn.functional.softmax(outputs.logits, dim=-1)

pred_idx = probs.argmax().item()
pred_label = id2label[pred_idx]
pred_prob = probs[0][pred_idx].item()

print("\n" + "="*100)
print(f"Исходный текст: «{text}»")
print(f"Предсказанная метка: {pred_label}")
print(f"Вероятность: {pred_prob:.4f}")
print("\nВсе вероятности:")
for i, p in enumerate(probs[0]):
    print(f"  {id2label[i]}: {p:.4f}")
print("="*100)

# Выбор лучшей модели (для инференса)